In [ ]:
import os
import io
from google.cloud import vision
from google.cloud.vision import types

## OCR 

In [ ]:
def detect_text(path):
    """Detects text in the file."""
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = types.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    print('Texts:')

    for text in texts:
        print('\n"{}"'.format(text.description))

In [ ]:
detect_text('img/alberto_dni.JPG')

## Faces detection

In [ ]:
import pylab as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np

In [ ]:
def detect_faces(path):
    """Detects faces in an image."""
    client = vision.ImageAnnotatorClient()
    

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = types.Image(content=content)

    response = client.face_detection(image=image)
    faces = response.face_annotations

    # Names of likelihood from google.cloud.vision.enums
    likelihood_name = ('UNKNOWN', 'VERY_UNLIKELY', 'UNLIKELY', 'POSSIBLE',
                       'LIKELY', 'VERY_LIKELY')
    print('Faces:')

    for face in faces:
        print('anger: {}'.format(likelihood_name[face.anger_likelihood]))
        print('joy: {}'.format(likelihood_name[face.joy_likelihood]))
        print('surprise: {}'.format(likelihood_name[face.surprise_likelihood]))

        vertices = (['({},{})'.format(vertex.x, vertex.y)
                    for vertex in face.bounding_poly.vertices])
        vertices_int = [(vertex.x,vertex.y) for vertex in face.bounding_poly.vertices]
        

        print('face bounds: {}'.format(','.join(vertices)))
    
    x = int(vertices_int[0][0])
    y = int(vertices_int[0][1])
    width = np.absolute(int(vertices_int[0][0]) - int(vertices_int[1][0]))
    height = np.absolute(int(vertices_int[0][1]) - int(vertices_int[2][1]))
    return x,y,width,height

In [ ]:
x,y,width,height = detect_faces('img/marian_drive.JPG')
im = np.array(Image.open('img/marian_drive.JPG'), dtype=np.uint8)

fig,ax = plt.subplots(1)

rect = patches.Rectangle((x,y),height,width,linewidth=1,edgecolor='r',facecolor='none')
ax.add_patch(rect)
ax.imshow(im)
plt.show()
    

In [ ]:
x,y,width,height = detect_faces('known_people/marian2.JPG')
im = np.array(Image.open('known_people/marian2.JPG'), dtype=np.uint8)

fig,ax = plt.subplots(1)

rect = patches.Rectangle((x,y),height,width,linewidth=1,edgecolor='r',facecolor='none')
ax.add_patch(rect)
ax.imshow(im)
plt.show()
    

In [ ]:
import face_recognition as fk
from skimage.transform import resize

In [ ]:
def load(path):return resize(fk.load_image_file(path), (600,800), mode='reflect')
def compare(im_array1,im_array2):
    try:
        encoding1 = fk.face_encodings(im_array1)[0]
        encoding2 = fk.face_encodings(im_array2)[0]
        return fk.compare_faces([encoding1], encoding2)
    except Exception as e:
        print("Error: %s" %e)

In [ ]:
marian,alberto = load('img/marian.JPG'), load('img/alberto.JPG')
dni_alberto,driver_marian = fk.load_image_file('img/alberto_dni.JPG'),fk.load_image_file('img/marian_drive.JPG')

In [ ]:
compare(driver_marian, marian)